In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deploy ADK Agent in AI Engine

This notebook provides a step-by-step to deploy an Agent Created using Agent Development Kit on Agent Engine (ReasonEngine on Vertex AI)

**Important**: This notebook consider that the Agent was built with ADK and the agent files are inside an agent folder and the dependencies in a file config.yaml

Folders structure (example): 
```
parent_folder/
    agent_folder/
        __init__.py
        agent.py
        config.yaml
        .env
    deploy_agent_engine.ipynb
```

config.yaml (example): 
```yaml
agent_name: 'agent_name'
agent_display_name: 'Agent Name'
agent_description: 'Useful agent to help users'

deploy:
  dependencies: ['google-cloud-aiplatform[agent_engines]', 'google-adk', 'cloudpickle', 'pydantic']
```


### Setup and Config

In [1]:
# Checking the google-adk and google-cloud-aiplatform versions
!pip freeze | grep google-adk
!pip freeze | grep google-cloud-aiplatform

google-adk==1.12.0
google-cloud-aiplatform==1.110.0


In [ ]:
# Authentication on gcloud (if necessary)
# !gcloud auth application-default login

In [2]:
# Basic Libraries
import os 
import vertexai
import yaml

# AI Engine on Vertex AI 
from vertexai import agent_engines

# Library for AI Engine with ADK
from vertexai.preview import reasoning_engines

# Just to view JSON response formatted
import json
from IPython.display import display,Markdown,JSON

# To load envvars dict from .env file
from dotenv import dotenv_values

In [3]:
# Load Agent Config
AGENT_DIR = "adk_bq_agent"

# Load environment variables from .env file from agent Directory 
from dotenv import load_dotenv
env_file = f'./{AGENT_DIR}/.env'
load_dotenv(env_file)

# Load config from agents (Params and dependencies for deploy)
with open(f'./{AGENT_DIR}/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# For Vertex AI SDK 
PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
LOCATION = os.environ.get("GOOGLE_CLOUD_LOCATION")
BUCKET = os.environ.get("GOOGLE_CLOUD_BUCKET")

### Instantiate Agent from Directory

In [4]:
# Importing Agent Module from AGENT_DIR folder
import importlib
agent_module = importlib.import_module(f"{AGENT_DIR.replace('/','.')}.agent")

# Instantiate the Assistant as an ADK App 
adk_agent = reasoning_engines.AdkApp(
    agent=agent_module.root_agent,
)

BigQuery context loaded successfully from 'bigquery_context.txt'


### Running Agent Local (Optional)

In [5]:
# Run a simple query
for event in adk_agent.stream_query(
    user_id="user",
    message="Hi, how can you help me?",
):
    pass

# Formatted output
display(Markdown(f"```json\n{json.dumps(event, indent=2,ensure_ascii=False)}\n```"))

```json
{
  "content": {
    "parts": [
      {
        "text": "I can help you by querying a BigQuery database. I can answer questions about customer profiles, financial accounts, transaction history, and customer propensity for new products or services. Just tell me what you need to know!\n\nFor example, I can tell you:\n* The names of customers with the highest account balances.\n* The average transaction value for a specific type of transaction.\n* Customers who are likely to accept certain product offers.\n* And much more, based on the available data!\n\nWhat would you like to know today?"
      }
    ],
    "role": "model"
  },
  "finish_reason": "STOP",
  "usage_metadata": {
    "candidates_token_count": 110,
    "candidates_tokens_details": [
      {
        "modality": "TEXT",
        "token_count": 110
      }
    ],
    "prompt_token_count": 7532,
    "prompt_tokens_details": [
      {
        "modality": "TEXT",
        "token_count": 7532
      }
    ],
    "total_token_count": 7642,
    "traffic_type": "ON_DEMAND"
  },
  "invocation_id": "e-3790626f-9a78-404e-8215-3c10465f4715",
  "author": "bigquery_agent",
  "actions": {
    "state_delta": {},
    "artifact_delta": {},
    "requested_auth_configs": {}
  },
  "id": "fab16857-4a7b-4aed-a3e6-3fe4f7223f7c",
  "timestamp": 1760028021.990443
}
```

### Deploy on Agent Engine

In [6]:
# Instantiate Vertex AI
vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=BUCKET,
)

In [7]:
## Retrieve all existent Agent Engine on your project
for agent in agent_engines.list():
    print(f"============================ \nAgent: {agent.display_name}\nResoruce Name: {agent.resource_name}\nCreated/updated at: {agent.update_time} \n\n" )

Agent: ADK BQ Agent
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/4745267885109149696
Created/updated at: 2025-09-25 17:02:34.211505+00:00 


Agent: ADK Artifact Agent
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/2918829537791836160
Created/updated at: 2025-09-24 17:40:47.932132+00:00 


Agent: ADK OAuth Agent
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/3497471721164767232
Created/updated at: 2025-08-27 19:24:10.973773+00:00 


Agent: Success Factors Agent
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/180992808071462912
Created/updated at: 2025-08-23 13:56:47.576121+00:00 


Agent: Moodle Data Agent
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/8056649462296084480
Created/updated at: 2025-08-21 14:32:27.457926+00:00 


Agent: Procurement Analytics
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/8961019766374

In [8]:
# Read Requirements for Agent from config file
# Usually this ['google-cloud-aiplatform[agent_engines]', 'google-adk', 'cloudpickle'] plus the packages that agent needs
requirements = config['deploy']['dependencies']
requirements

['google-cloud-aiplatform[agent_engines,adk]',
 'google-adk',
 'cloudpickle',
 'pydantic',
 'python-dotenv',
 'pyyaml',
 'google-cloud-bigquery']

In [9]:
# Extra packages from agent folder (This is all .py files inside Agent Directory)
extra_packages = [AGENT_DIR]
extra_packages

['adk_bq_agent']

In [10]:
# Load Variables on env_vars dict to be used when creating the Agent
env_vars = dotenv_values(dotenv_path=env_file)

# Remove GCP variables (this variables already are defined in Agent Engine and are reserved)
keys_to_remove = [
    "GOOGLE_GENAI_USE_VERTEXAI",
    "GOOGLE_CLOUD_PROJECT",
    "GOOGLE_CLOUD_LOCATION",
    "GOOGLE_CLOUD_BUCKET"
]

for key in keys_to_remove:
    env_vars.pop(key, None)


In [11]:
# Deploy the Agent on AI Engine (This takes a few minutes)
remote_agent = agent_engines.create(
    agent_engine = adk_agent,             # The Agent instantiated as ADK agent
    requirements=requirements,            # Requirements file
    extra_packages=extra_packages,        # Extra packages
    display_name=config['agent_display_name'],    # Display name  
    description=config['agent_description'],     # Description
    env_vars=env_vars                     # Env Vars dict
)

Identified the following requirements: {'pydantic': '2.11.7', 'google-cloud-aiplatform': '1.110.0', 'cloudpickle': '3.1.1'}
The final list of requirements: ['google-cloud-aiplatform[agent_engines,adk]', 'google-adk', 'cloudpickle', 'pydantic', 'python-dotenv', 'pyyaml', 'google-cloud-bigquery']
Using bucket speca-argolis-adk-agents
Wrote to gs://speca-argolis-adk-agents/agent_engine/agent_engine.pkl
Writing to gs://speca-argolis-adk-agents/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://speca-argolis-adk-agents/agent_engine/dependencies.tar.gz
failed to generate schema for async_add_session_to_memory: `async_add_session_to_memory` is not fully defined; you should define `Session`, then call `async_add_session_to_memory.model_rebuild()`.

For further information visit https://errors.pydantic.dev/2.11/u/class-not-fully-defined
Creating AgentEngine
Create AgentEngine backing LRO: projects/809256159830/locations/us-central1/reasoningEngines/909356

### Test Remote Agent on Agent Engine

In [12]:
## Retrieve all existent Agent Engine resource.names (Agents)
# To confirm new agent was deployed
for agent in agent_engines.list():
    print(f"============================ \nAgent: {agent.display_name}\nResoruce Name: {agent.resource_name}\nCreated/updated at: {agent.update_time} \n\n" )

Agent: ADK BQ Agent
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/9093563694079541248
Created/updated at: 2025-10-09 16:45:31.835380+00:00 


Agent: ADK BQ Agent
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/4745267885109149696
Created/updated at: 2025-09-25 17:02:34.211505+00:00 


Agent: ADK Artifact Agent
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/2918829537791836160
Created/updated at: 2025-09-24 17:40:47.932132+00:00 


Agent: ADK OAuth Agent
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/3497471721164767232
Created/updated at: 2025-08-27 19:24:10.973773+00:00 


Agent: Success Factors Agent
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/180992808071462912
Created/updated at: 2025-08-23 13:56:47.576121+00:00 


Agent: Moodle Data Agent
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/8056649462296084480
Cr

In [13]:
# Confirm that "remote_agent" is pointing to your new agent
print(f"=================== Remote Agent ============================ \n\
 Name: {remote_agent.display_name}\n\
 Resoruce Name: {remote_agent.resource_name}\n\
 Created/updated at: {remote_agent.update_time} \n\n" )

=================== Remote Agent ============================ 
 Name: ADK BQ Agent
 Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/9093563694079541248
 Created/updated at: 2025-10-09 16:45:31.835380+00:00 




In [16]:
# Run a simple query
for remote_event in remote_agent.stream_query(
    user_id="user",
    message="Hi, what you can do form me?",
):
    display(JSON(remote_event,expanded=True)) 

In [17]:
# Formatted final output
display(Markdown(f"```json\n{json.dumps(remote_event, indent=2,ensure_ascii=False)}\n```"))

NameError: name 'remote_event' is not defined